In [ ]:
# run to pretty-print results
import numpy as np

np.set_printoptions(precision=5, suppress=True)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Using average-minimum-distance (amd)

The amd package contains functions for caluclating and comparing AMDs/PDDs, as well as .cif reading functionality provided by either ase or ccdc (license required).

If not installed already, get amd via pip:
```shell
$ pip install average-minimum-distance
```

To read .cifs it is recommended you also pip install ase, the default back end .cif reader. If you have ccdc installed and a valid license, it can be used instead. ccdc also enables searching the CSD by refcode. 

**Before running any cells below, import amd by running**

In [ ]:
import amd

## Reading cifs

To read cifs with amd, create an ```amd.CifReader``` object and pass the path to the cif as below. (The .cifs in this notebook can be found in the tests folder of this project). The CifReader will try to extract the correct information; if a structure cannot be read by default it is skipped by the reader and a warning is printed. This returns an iterator which can be looped over to get all structures from a file. It yields ```PeriodicSet``` objects, which can be handed directly to the AMD/PDD calculator functions below. ```PeriodicSet```s basically have only 3 attributes, a ```name```, ```motif``` and ```cell```.

In [ ]:
# one cif, many structures

reader = amd.CifReader('T2_experimental.cif')
for periodic_set in reader:
    print(periodic_set.name, periodic_set.motif.shape[0])   # print name & number of motif points
    periodic_set.cell                                       # access unit cell

# if you don't care about lazy reading and just want a list of structures,
exp_structures = list(amd.CifReader('T2_experimental.cif'))

Below, setting ```remove_hydrogens``` to True will ignore all Hydrogen atoms in the file. The CifReader has a few optional arguments, but this is the only stable one and others (like ```heaviest_component``` below) should be only be changed with caution.

In [ ]:
# one cif, one structure

gamma = list(amd.CifReader('T2_gamma.cif', remove_hydrogens=True))[0]

T2_alpha.cif has a solvent molecule with disorder. Setting ```heaviest_component``` to True (ccdc only) will attempt to remove the solvent by only extracting the heaviest molecule in the asymmetric unit. Use with caution; if for example the asymmetric unit is already expanded in your .cif or you have a co-crystal this results in unintended behaviour.

In [ ]:
# an odd case where we need ccdc. Will error if not installed or licensed

reader = amd.CifReader('T2_alpha.cif', reader='ccdc', heaviest_component=True)
alpha = list(reader)[0]
print(alpha)

## Calculating AMDs and PDDs

The main calculator functions ```amd.amd``` and ```amd.pdd``` accept two arguments, a periodic set and an integer ```k```. The periodic set can be either a ```PeriodicSet``` object as given by a CifReader, or a pair of numpy arrays (motif, cell).

In [ ]:
k = 100

# one AMD from a .cif with one structure
gamma = list(amd.CifReader('T2_gamma.cif'))[0]
gamma_amd = amd.amd(gamma, k) 
print(gamma_amd)

# list of amds from a .cif with many structures
exp_structures = amd.CifReader('T2_experimental.cif')
experimental_amds = [amd.amd(periodic_set, k) for periodic_set in exp_structures] # list of AMDs

In [ ]:
k = 100

# the pdd interface is the same as amd
gamma = list(amd.CifReader('T2_gamma.cif'))[0]
gamma_pdd = amd.pdd(gamma, k)

print(gamma_pdd)

In [ ]:
import numpy as np

k = 100

# from a tuple (motif, cell) of numpy arrays
motif = np.array([[0,0,0]]) # one point at the origin
cell = np.identity(3)       # unit cell = identity (cube with unit edges)
cubic_pdd = amd.pdd((motif, cell), k)

print(cubic_pdd)

## Comparing AMDs and PDDs

Two AMDs are typically compared with the l-infinity or Chebyshev metric. Comparing AMDs should be done with ```amd.compare``` (which essentially wraps scipy's cdist for now). It takes two arguments ```reference``` and ```comparison```, either of which can be a single AMD, a list of AMDs, or a numpy array with AMDs in the rows. It returns a distance matrix where the (i,j)-th entry is the AMD-distance between ```reference[i]``` and ```comparison[j]```. If single AMDs are passed it will still return a 2D matrix, so if ```amd_1, amd_2``` are AMDs then ```amd.compare(amd_1, amd_2)[0][0]``` is the distance between them.

In [ ]:
import numpy as np

k = 100

gamma = list(amd.CifReader('T2_gamma.cif'))[0]
gamma_amd = amd.amd(gamma, k)

exp_structures = list(amd.CifReader('T2_experimental.cif'))
experimental_amds = [amd.amd(periodic_set, k) for periodic_set in exp_structures]

# compare T2-gamma to the other experimental structures
distance_matrix = amd.compare(gamma_amd, experimental_amds)
print(distance_matrix)

# this will sort the output by distance and print the closest structures to T2_gamma in order
for i in np.argsort(distance_matrix[0]):
    print(exp_structures[i].name, distance_matrix[0][i])

In [ ]:
# compare a set of AMDs pairwise
distance_matrix = amd.compare(experimental_amds, experimental_amds)
print(distance_matrix)

In [ ]:
import numpy as np

motif = np.array([[0,0,0]]) # one point at the origin
cell = np.identity(3)       # unit cell = identity
cubic_amd = amd.amd((motif, cell), k)
cubic_amd_ = amd.amd(cubic, k)
print(amd.compare(cubic_amd, cubic_amd_))

To compare PDDs, use ```amd.emd``` which accepts excatly two PDDs as arguments and returns one number. 

In [ ]:
k = 1000

# construct cubic lattice manually
motif = np.array([[0,0,0]])
cell = np.identity(3)      
cubic_pdd = amd.pdd((motif, cell), k)

# read same structure from cif
cubic_pdd_cif = amd.pdd(list(amd.CifReader('cubic.cif'))[0], k)

# they should be identical (distance 0)
print(amd.emd(cubic_pdd, cubic_pdd_cif))

## Extras

### Reading by CSD refcode (ccdc only)

If you have the ccdc package installed, the ```CSDReader``` object lets you read entries from the CSD. Instead of a path to a file it accepts a list of CSD refcodes. It takes the same optional parameters as ```CifReader``` (except ```reader```), e.g. ```remove_hydrogens```. It has another optional parameter ```families``` which reads in families of structures from the base refcode:

In [ ]:
import amd

# get DEBXIT and ACSALA01
print(list(amd.CSDReader(['DEBXIT', 'ACSALA01'])))

# get both families (anything like DEBXITXX or ACSALAXX)
print(list(amd.CSDReader(['DEBXIT', 'ACSALA'], families=True)))

# Using ase directly to read .cifs

In [ ]:
import ase.io

atoms = ase.io.read('T2_gamma.cif')             # returns one atoms object
iterator = ase.io.iread('T2_experimental.cif')  # yields atoms objects

print(atoms)
print(atoms.get_cell().array)                # Cartesian unit cell
print(atoms.get_positions(wrap=True).shape)  # Cartesian motif (wrap positions into cell)